In [1]:
import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F

class TwoPathConv(nn.Module):
    def __init__(self):
        super(TwoPathConv, self).__init__()
        self.upper_layer1 = nn.Sequential(
            nn.Conv2d(4,64,7),
            nn.ReLU(),
            nn.MaxPool2d((4,4),stride = 1)
        )
        self.upper_layer2 = nn.Sequential(
            nn.Conv2d(64,64,3),
            nn.ReLU(),
            nn.MaxPool2d((2,2),stride = 1)
        )
        self.under_layer1 = nn.Sequential(
            nn.Conv2d(4,160,13),
            nn.ReLU()
        )
        self.final_layer = nn.Conv2d(224,5,21)
        
    def forward(self, x):
        upper_x = self.upper_layer2(self.upper_layer1(x))
        under_x = self.under_layer1(x)
        #upper_x = F.max_pool2d(F.relu(self.upper_conv1(x)), (4, 4),stride = 1)
        #upper_x = F.max_pool2d(F.relu(self.upper_conv2(upper_x)), (2, 2), stride = 1)
        #under_x = F.relu(self.under_conv1(x))
        final_x = torch.cat((under_x, upper_x), 1)
        out = self.final_layer(final_x)
        return out
        
net = TwoPathConv()
print(net)

x = Variable(torch.randn(1,4,33,33), requires_grad = True)
y_pred = net.forward(x)
print(y_pred)
#y_pred = y_pred.data.resize_(1,5)
#y_pred = Variable(y_pred,requires_grad = True)
#print(y_pred.size())
#print(y_pred.view)

TwoPathConv (
  (upper_layer1): Sequential (
    (0): Conv2d(4, 64, kernel_size=(7, 7), stride=(1, 1))
    (1): ReLU ()
    (2): MaxPool2d (size=(4, 4), stride=(1, 1), dilation=(1, 1))
  )
  (upper_layer2): Sequential (
    (0): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1))
    (1): ReLU ()
    (2): MaxPool2d (size=(2, 2), stride=(1, 1), dilation=(1, 1))
  )
  (under_layer1): Sequential (
    (0): Conv2d(4, 160, kernel_size=(13, 13), stride=(1, 1))
    (1): ReLU ()
  )
  (final_layer): Conv2d(224, 5, kernel_size=(21, 21), stride=(1, 1))
)
Variable containing:
(0 ,0 ,.,.) = 
 -0.2623

(0 ,1 ,.,.) = 
  0.2611

(0 ,2 ,.,.) = 
 -0.5541

(0 ,3 ,.,.) = 
 -0.3231

(0 ,4 ,.,.) = 
 -0.0795
[torch.FloatTensor of size 1x5x1x1]



In [2]:
#get the training set for phase 1
f=open("trainval-balanced.txt", "r")
content=f.readlines()
data=[]
data_train_phase1=[]
i=0
from random import shuffle
shuffle(content)
for line in content:
    no_n_line=line[0:len(line)-1]
    item=no_n_line.split(" ")
    data.append([])
    data[i].append(item[0])
    data[i].append(int(item[1]))
    data[i].append(int(item[2]))
    data[i].append(int(item[3]))
    data[i].append(int(item[4]))
    data_train_phase1.append(data[i])
    i += 1
f.close()

print ("phase 1 data preparation process completed.")

#get the training set for phase 2 and the validation set
data_val=[]
f_in=open("trainval.txt", "r")
content=f_in.readlines()
data=[]
data_val=[]
data_train_phase2=[]
i=0
shuffle(content)
for line in content:
    no_n_line=line[0:len(line)-1]
    item=no_n_line.split(" ")
    data.append([])
    data[i].append(item[0])
    data[i].append(int(item[1]))
    data[i].append(int(item[2]))
    data[i].append(int(item[3]))
    data[i].append(int(item[4]))
    if i%30000==0:
        data_val.append(data[i])
    else:
        data_train_phase2.append(data[i])
    i += 1
f_in.close()
print ("phase 2 data preparation process completed.")


import h5py
f = h5py.File('training.h5','r')

phase 1 data preparation process completed.
phase 2 data preparation process completed.


In [3]:
print (len(data))
print (len(data_val))
print (len(data_train_phase1))
print (len(data_train_phase2))

24778697
826
8998296
24777871


In [4]:
print (torch.cuda.is_available())

True


In [5]:
import torch.nn.init as ninit
def init_net(net):
    for param in net.parameters():
        ninit.uniform(param.data, a=-5e-3, b=5e-3)

In [6]:
import numpy as np
def create_batch_phase1(start, batch_size):
    X_batch= None
    y_batch= None
    if start>=len(data_train_phase1):
        return X_batch, y_batch
    end=start+batch_size
    if end>=len(data_train_phase1):
        end=len(data_train_phase1)
    X_batch = []
    y_batch = []
    for index in range(start, end):
        case, x, y, z, l = data_train_phase1[index]
        case1 = case[:2]
        case2 = case[3:]
        X_batch.append(f[case1][case2][:, x-16:x+17, y-16:y+17, z])
        y_batch.append(l)
    X_batch = torch.from_numpy(np.array(X_batch))
    y_batch = torch.from_numpy(np.array(y_batch))
    return X_batch, y_batch

def create_batch_phase2(start, batch_size):
    X_batch= None
    y_batch= None
    if start>=len(data_train_phase2):
        return X_batch, y_batch
    end=start+batch_size
    if end>=len(data_train_phase2):
        end=len(data_train_phase2)
    X_batch = []
    y_batch = []
    for index in range(start, end):
        case, x, y, z, l = data_train_phase2[index]
        case1 = case[:2]
        case2 = case[3:]
        X_batch.append(f[case1][case2][:, x-16:x+17, y-16:y+17, z])
        y_batch.append(l)
    X_batch = torch.from_numpy(np.array(X_batch))
    y_batch = torch.from_numpy(np.array(y_batch))
    return X_batch, y_batch

def create_val(batch_mask):
    X_val=[]
    y_val=[]
    for i in range(len(batch_mask)):
        case, x, y, z, l = data_val[batch_mask[i]]
        case1 = case[:2]
        case2 = case[3:]
        X_val.append(f[case1][case2][:, x-16:x+17, y-16:y+17, z])
        y_val.append(l)
    X_val = torch.from_numpy(np.array(X_val))
    y_val = torch.from_numpy(np.array(y_val))
    return X_val, y_val

In [12]:
import threading
from threading import Thread
class DataThread(Thread):
    def __init__(self,func,args,name=''):
        threading.Thread.__init__(self)
        self.name=name
        self.func=func
        self.args=args
    
    def run(self):
        index, batch_size=self.args
        self.X_batch, self.y_batch=self.func(index, batch_size)

class GetTrainDataPhase1():
    def __init__(self, index, batch_size, thread_num=4):
        self.threads=[]
        batch_len=batch_size//thread_num
        self.X_batch=[]
        self.y_batch=[]
        start=(index*batch_size)%len(data_train_phase1)
        for i in range(thread_num):
            start_index=start+i*batch_len
            t=DataThread(create_batch_phase1, (start_index, batch_len))
            self.threads.append(t)
    
    def run(self):
        for i in range(len(self.threads)):
            self.threads[i].start()
        for i in range(len(self.threads)):
            self.threads[i].join()
            self.X_batch.append(self.threads[i].X_batch)
            self.y_batch.append(self.threads[i].y_batch)
        X_batch=torch.cat(self.X_batch)
        y_batch=torch.cat(self.y_batch)
        return X_batch, y_batch
            
    

In [7]:
import multiprocessing
from multiprocessing import Queue

def Process1(index, batch_size, nprocs=4):
    def worker(index, batch_size, out_q):
        if start>=len(data_train_phase1):
            return
        end=start+batch_size
        if end>=len(data_train_phase1):
            end=len(data_train_phase1)
        X_batch = []
        y_batch = []
        for index in range(start, end):
            case, x, y, z, l = data_train_phase1[index]
            case1 = case[:2]
            case2 = case[3:]
            X_batch.append(f[case1][case2][:, x-16:x+17, y-16:y+17, z])
            y_batch.append(l)
        X_batch = np.array(X_batch)
        y_batch = np.array(y_batch)
        out_q.put((X_batch, y_batch))
    
    out_q=Queue()
    procs=[]
    batch_len=batch_size//nprocs
    start=(index*batch_size)%len(data_train_phase1)
    for i in range(nprocs):
        start_index=start+i*batch_len
        p = multiprocessing.Process(
                target=worker,
                args=(start_index, batch_len, out_q))
        procs.append(p)
        p.start()
    X_list=[]
    y_list=[]
    for i in range(nprocs):
        X_frac, y_frac=out_q.get()
        X_list.append(torch.from_numpy(X_frac))
        y_list.append(torch.from_numpy(y_frac))
    for p in procs:
        p.join()
    X_batch=torch.cat(X_list)
    y_batch=torch.cat(y_list)
    return X_batch, y_batch

def Process2(index, batch_size, nprocs=4):
    def worker(index, batch_size, out_q):
        if start>=len(data_train_phase2):
            return
        end=start+batch_size
        if end>=len(data_train_phase2):
            end=len(data_train_phase2)
        X_batch = []
        y_batch = []
        for index in range(start, end):
            case, x, y, z, l = data_train_phase2[index]
            case1 = case[:2]
            case2 = case[3:]
            X_batch.append(f[case1][case2][:, x-16:x+17, y-16:y+17, z])
            y_batch.append(l)
        X_batch = np.array(X_batch)
        y_batch = np.array(y_batch)
        out_q.put((X_batch, y_batch))
    
    out_q=Queue()
    procs=[]
    batch_len=batch_size//nprocs
    start=(index*batch_size)%len(data_train_phase1)
    for i in range(nprocs):
        start_index=start+i*batch_len
        p = multiprocessing.Process(
                target=worker,
                args=(start_index, batch_len, out_q))
        procs.append(p)
        p.start()
    X_list=[]
    y_list=[]
    for i in range(nprocs):
        X_frac, y_frac=out_q.get()
        X_list.append(torch.from_numpy(X_frac))
        y_list.append(torch.from_numpy(y_frac))
    for p in procs:
        p.join()
    X_batch=torch.cat(X_list)
    y_batch=torch.cat(y_list)
    return X_batch, y_batch

In [ ]:
import time
import numpy as np
num_train=len(data_train_phase1)
num_val=len(data_val)
batch_size=512
val_size=num_val
num_epoch=2.0
num_times=int(float(num_train)/batch_size*num_epoch)
print (num_times)
learning_rate = 5e-3
reg=5e-5
net = TwoPathConv()
print(net)
init_net(net)
net.cuda(0)

#create validation set
val_mask=np.random.choice(num_val, val_size)
X_val, y_val = create_val(val_mask)
X_val = Variable(X_val.cuda(0), requires_grad=False)

prev_time = time.clock()
optimizer = torch.optim.SGD(net.parameters(), lr=learning_rate, momentum=0.9, weight_decay=reg)
net.zero_grad()
for i in range(21):  # loop over the dataset multiple times
    X_batch=None
    batch_time=time.clock()
    X_batch, y_batch=Process1(i, batch_size)
    print (X_batch.size())
    print("batch_time: "+ str(time.clock()-batch_time))
    X_batch, y_batch = Variable(X_batch.cuda(0)), Variable(y_batch.cuda(0), requires_grad = False)
    y_pred = net.forward(X_batch)
    y_pred = y_pred.view(-1,5)
    loss = F.cross_entropy(y_pred, y_batch)
    loss.backward()
    optimizer.step()
    net.zero_grad()
    if i % 500 == 0:
        print ("")
        print ('time used %.3f' % (time.clock()-prev_time))
        print (str(float(i)/num_times*100)+"% completed")
        print (loss)
        y_val_pred=net.forward(X_val)
        y_val_pred=y_val_pred.view(-1,5)
        useless, predicted=torch.max(y_val_pred.data, 1)
        correct = (predicted == y_val.cuda(0)).sum()
        print('Validation accuracy:', float(correct)/val_size)
print ("phase1 successfully trained!")
#torch.save(net.state_dict(), "premature_net.txt")
print ("phase1 successfully saved!")

35149
TwoPathConv (
  (upper_layer1): Sequential (
    (0): Conv2d(4, 64, kernel_size=(7, 7), stride=(1, 1))
    (1): ReLU ()
    (2): MaxPool2d (size=(4, 4), stride=(1, 1), dilation=(1, 1))
  )
  (upper_layer2): Sequential (
    (0): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1))
    (1): ReLU ()
    (2): MaxPool2d (size=(2, 2), stride=(1, 1), dilation=(1, 1))
  )
  (under_layer1): Sequential (
    (0): Conv2d(4, 160, kernel_size=(13, 13), stride=(1, 1))
    (1): ReLU ()
  )
  (final_layer): Conv2d(224, 5, kernel_size=(21, 21), stride=(1, 1))
)
torch.Size([512, 4, 33, 33])
batch_time: 0.41813700000000154

time used 0.937
0.0% completed
Variable containing:
 1.6081
[torch.cuda.FloatTensor of size 1 (GPU 0)]

Validation accuracy: 0.17554479418886199
torch.Size([512, 4, 33, 33])
batch_time: 0.43669999999997344
torch.Size([512, 4, 33, 33])
batch_time: 0.4432909999999879
torch.Size([512, 4, 33, 33])
batch_time: 0.4394990000000121
torch.Size([512, 4, 33, 33])
batch_time: 0.50867799999997